In [1]:
import myproject
import pandas as pd
import json
import numpy as np

exp ['always5']
example.py {'fillna': [[3, 'fillVal', 'type', 'integer']], 'dropcol': [None], 'onehot': [None], 'groupby': [[3, 'colMap', 'colEnum', ['null', 'sum', 'mean', 'median', 'count']]]}


In [2]:
df = pd.read_csv('./problem.csv')
df2 = pd.read_csv('../dcf-server/sample_data/yellow_tripdata_2021-02.csv')

In [3]:
w = myproject.ExampleWidget(df=df)
w

ExampleWidget(command_config={'commandPatterns': {'fillna': [[3, 'fillVal', 'type', 'integer']], 'dropcol': [N…

In [5]:
small_command_config = {'commandPatterns': {'fillna': [[3, 'fillVal', 'type', 'integer']],
  'dropcol': [None],
  'onehot': [None]},
 'commandDefaults': {'fillna': [{'symbol': 'fillna'},
   {'symbol': 'df'},
   'col',
   8],
  'dropcol': [{'symbol': 'dropcol'}, {'symbol': 'df'}, 'col'],
  'onehot': [{'symbol': 'onehot'}, {'symbol': 'df'}, 'col']}}
w.command_config = small_command_config

In [7]:
orig_command_config = {'commandPatterns': {'fillna': [[3, 'fillVal', 'type', 'integer']],
  'dropcol': [None],
  'onehot': [None],
  'groupby': [[3,
    'colMap',
    'colEnum',
    ['null', 'sum', 'mean', 'median', 'count']]]},
 'commandDefaults': {'fillna': [{'symbol': 'fillna'},
   {'symbol': 'df'},
   'col',
   8],
  'dropcol': [{'symbol': 'dropcol'}, {'symbol': 'df'}, 'col'],
  'onehot': [{'symbol': 'onehot'}, {'symbol': 'df'}, 'col'],
  'groupby': [{'symbol': 'groupby'}, {'symbol': 'df'}, 'col', {}]}}

In [ ]:
df2

In [ ]:
w.transform_error

In [ ]:
w2 = myproject.ExampleWidget(df2)
w2

In [ ]:
w

In [ ]:
w.transformed_df = json.loads(df.to_json(orient='table', indent=2))

In [ ]:
w.transformed_df.on?

In [ ]:
from myproject.dcf import 

In [ ]:
w.value

In [ ]:
w.value2

In [ ]:
w.commands

In [ ]:
assert w.value == 'Hello World'

In [ ]:
w.value = "asdfasdf"